# MDPs and Q-learning On "Ice" (60 points possible)

In this assignment, we’ll revisit Markov Decision Processes while also trying out Q-Learning, the reinforcement learning approach that associates utilities with attempting actions in states.
The problem that we’re attempting to solve is the following:

1.  There is a grid of spaces in a rectangle.  Each space can contain a pit (negative reward), gold (positive reward), or nothing.
2.  The rectangle is effectively surrounded by walls, so anything that would move you outside the rectangle, instead moves you to the edge of the rectangle.
3.  The floor is icy.  Any attempt to move in a cardinal direction results in moving a somewhat random number of spaces in that direction.  The exact probabilities of moving each number of spaces are given in the problem description.  (If you slide too far, see rule #2.)
4.  Landing on a pit or gold effectively “ends the run,” for both a Q learner and an agent later trying out the policy.  It’s game over.  (To simulate this during Q learning, set all Q values for the space to equal its reward, then start over from a random space.)  Note that it's still possible to slide past a pit or gold - this doesn't end the run.

A sample input looks like this:


In [1]:
sampleMDP = """0.7 0.2 0.1
- - P - -
- - G P -
- - P - -
- - - - -"""


The first line says that the probabilities of moving one, two, or three spaces in the direction of movement are 0.7, 0.2, and 0.1.   The rest is a map of the environment, where a dash is an empty space, P is a pit, and G is gold.

Your job is to finish the code below for mdp_solve() and q_solve().  These take a problem description like the one pictured above, and return a policy giving the recommended action to take in each empty square (U=up, R=right, D=down, L=left).

**1, 17 points)**  mdp_solve() should use value iteration and the Bellman equation.  ITERATIONS will refer to the number of complete passes you perform over all states.  You can initialize the utilities to the rewards of each state.  Don’t update the rewards spaces from their initial rewards; since they end the trial, they have no future utility.  Don't update utilities in-place as you iterate through them, but create a fresh array of utilities with each pass, in order to avoid biasing moves in the directions that have already been updated.

**2, 24 points)**  q_solve() will run ITERATIONS trials in which a learner starts in a random empty square and moves until it hits a pit or gold, in which case, the trial is over.  (If it was randomly dropped into gold or a pit, the trial is immediately over.)  The learner moves by deciding randomly whether to choose a random direction (with probability EXPLORE_PROB) or move according to the best Q-value of its current square (otherwise).  Simulate the results of the move on slippery ice to determine where the learner ended up - then apply the Q-learning equation given in lecture and the textbook.  (There are multiple Q-learning variants out there, so try to use the equations and practices described in lecture instead of using other sources, to avoid confusion.)

The fact that a trial ends immediately on finding gold or a pit means that we want to handle those spaces in a special way.  Normally Q values are updated on moving to the next state, but we won’t see any next state in these cases.  So, to handle this, when the agent discovers one of these rewards, set all the Q values for that space to the associated reward before quitting the trial.  So, for example, if gold is worth 100 and it’s discovered in square x, Q(x,UP) = 100, Q(x,RIGHT) = 100, Q(x, DOWN) = 100, and Q(x, LEFT) = 100.  There’s no need to apply the rest of the Q update equation when the trial is ending, because that’s all about future rewards, and there’s no future when the trial is ending.  But now the spaces that can reach that space will evaluate themselves appropriately.  (Before being "discovered," the square should have no utility.)

You should use the GOLD_REWARD, PIT_REWARD, LEARNING_RATE, and DISCOUNT_FACTOR constants at the top of the code box below.

Q-learning involves a lot of randomness and some arbitrary decisions when breaking ties, so two implementations can both be correct but recommend slightly different policies in the end, even if they have the same starting random seed.  While we provide some helpful premade maps below, your main guide for debugging will be common sense in deciding whether the policy created by your agent makes sense -- ie, agents following the policy will get gold without taking unnecessary risks.

In [2]:
""" "MDPs on Ice - Assignment 5"""

import random
import copy

GOLD_REWARD = 250.0
PIT_REWARD = -150.0
DISCOUNT_FACTOR = 0.8
EXPLORE_PROB = 0.2 # for Q-learning
LEARNING_RATE = 0.01
ITERATIONS = 20000
MAX_MOVES = 1000
ACTIONS = 4
UP = 0
RIGHT = 1
DOWN = 2
LEFT = 3
MOVES = ['U', 'R', 'D', 'L']

# Fixed random number generator seed for result reproducibility --
# don't use a random number generator besides this to match sol
random.seed(340)

class Problem:
    """Represents the physical space, transition probabilities, reward locations, and approach

    ...in short, the info in the problem string

    Attributes:
        move_probs (List[float]):  probabilities of going 1,2,3 spaces
        map (List[List(string)]]:  "-" (safe, empty space), "G" (gold), "P" (pit)

    String format consumed looks like
    0.7 0.2 0.1   [probability of going 1, 2, 3 spaces]
    - - - - - - P - - - -   [space-delimited map rows]
    - - G - - - - - P - -   [G is gold, P is pit]

    You can assume the maps are rectangular, although this isn't enforced
    by this constructor.
    """

    def __init__(self, probstring):
        """ Consume string formatted as above"""
        self.map = []
        for i, line in enumerate(probstring.splitlines()):
            if i == 0:
                self.move_probs = [float(s) for s in line.split()]
            else:
                self.map.append(line.split())

    def solve(self, iterations, use_q):
        """ Wrapper for MDP and Q solvers.

        Args:
            iterations (int):  Number of iterations (but these work differently for the two solvers)
            use_q (bool):  False means use MDP value iteration, true means use Q-learning
        Returns:
            A Policy, in either case (what to do in each square; see class below)
        """

        if use_q:
            return q_solve(self, iterations)
        return mdp_solve(self, iterations)

class Policy:
    """ Abstraction on the best action to perform in each state.

    This is a string list-of-lists map similar to the problem input, but a character gives the best
    action to take in each non-reward square (see MOVES constant at top of file).
    """

    def __init__(self, problem):
        """Args:

        problem (Problem):  The MDP problem this is a policy for
        """
        self.best_actions = copy.deepcopy(problem.map)

    def __str__(self):
        """Join the characters in the policy into one big space-separated, multline string"""
        return '\n{}\n'.format('\n'.join([' '.join(row) for row in self.best_actions]))

def roll_steps(move_probs, row, col, move, rows, cols):
    """Calculates the new coordinates that result from a move.

    Includes the "roll of the dice" for transition probabilities and checking arena boundaries.

    Helper for try_policy and q_solve - probably useful in your Q-learning implementation.

    Args:
        move_probs (List[float]):  Transition probabilities for the ice (from problem)
        row, col (int, int):  location of agent before moving
        move (string):  The direction of move as a MOVES character (not an int constant!)
        rows, cols (int, int):  number of rows and columns in the map

    Returns:
        new_row, new_col (int, int):  The new row and column after moving
    """
    displacement = 1
    total_prob = 0
    move_sample = random.random()
    for p, prob in enumerate(move_probs):
        total_prob += prob
        if move_sample <= total_prob:
            displacement = p+1
            break
    # Handle "slipping" into edge of map
    new_row = row
    new_col = col
    if not isinstance(move, str):
        print("Warning: roll_steps wants str for move, got a different type")
    if move == "U":
        new_row -= displacement
        if new_row < 0:
            new_row = 0
    elif move == "R":
        new_col += displacement
        if new_col >= cols:
            new_col = cols-1
    elif move == "D":
        new_row += displacement
        if new_row >= rows:
            new_row = rows-1
    elif move == "L":
        new_col -= displacement
        if new_col < 0:
            new_col = 0
    return new_row, new_col


def try_policy(policy, problem, iterations):
    """Returns average utility per move of the policy.

    Average utility is as measured by "iterations" random drops of an agent onto empty
    spaces, running until gold, pit, or time limit MAX_MOVES is reached.

    Doesn't necessarily play a role in your code, but you can try policies this
    way

    Args:
        policy (Policy):  the policy the agent is following
        problem (Problem):  the environment description
        iterations (int):  the number of random trials to run
    """
    total_utility = 0
    total_moves = 0
    for _ in range(iterations):
        # Resample until we have an empty starting square
        while True:
            row = random.randrange(0, len(problem.map))
            col = random.randrange(0, len(problem.map[0]))
            if problem.map[row][col] == "-":
                break
        for moves in range(MAX_MOVES):
            total_moves += 1
            policy_rec = policy.best_actions[row][col]
            # Take the move - roll to see how far we go, bump into map edges as necessary
            row, col = roll_steps(problem.move_probs, row, col, policy_rec, \
                                  len(problem.map), len(problem.map[0]))
            if problem.map[row][col] == "G":
                total_utility += GOLD_REWARD
                break
            if problem.map[row][col] == "P":
                total_utility += PIT_REWARD
                break
    return total_utility / total_moves


#helper function returns the new coordinates of a move
#accounts for cases where the distance goes off map-- insteas returns endge position
def get_pos(row,col,move,distance,problem):
    new_row = row
    new_col = col
    rows = len(problem.map)
    cols = len(problem.map[0])

    if move == "U":
        new_row -= distance
        if new_row < 0:
            new_row = 0
    elif move == "R":
        new_col += distance
        if new_col >= cols:
            new_col = cols-1
    elif move == "D":
        new_row += distance
        if new_row >= rows:
            new_row = rows-1
    elif move == "L":
        new_col -= distance
        if new_col < 0:
            new_col = 0
    return new_row, new_col

#Utility update function
def updateUtil(moves, problem,rewards,utils,r,c):
    best_u = -1000
    best_m = -1
    #Iterate through all possible moves
    for move in moves:
        curr = 0
        step = 1
        #Iterate through all possible distances and calculate expected utility for move
        while step <= len(problem.move_probs):
            new_r, new_c = get_pos(r,c,MOVES[move],step,problem)
            curr+= (problem.move_probs[step-1] * (DISCOUNT_FACTOR * utils[new_r][new_c]+rewards[new_r][new_c]))
            step+=1

        #Only keep move with best utility- thats the one which represents the square
        if curr> best_u:
            best_u = curr
            best_m = move


    return best_u, best_m



def mdp_solve(problem, iterations):
    """ Perform value iteration for the given number of iterations on the MDP problem.

    Here, the squares with rewards can be initialized to the reward values, since value iteration
    assumes complete knowledge of the environment and its rewards.

    Args:
        problem (Problem):  description of the environment
        iterations (int):  number of complete passes over the utilities
    Returns:
        a Policy (though you may design this to return utilities as a second return value)
    """
    r = len(problem.map)
    c = len(problem.map[0])
    utils = []
    rewards = []

    #Initializing utils and rewards before start
    for i in range(r):
        r_row = []
        for j in range(c):
            if problem.map[i][j]=="G":
                r_row.append(GOLD_REWARD)
            elif problem.map[i][j]=="P":
                r_row.append(PIT_REWARD)
            else:
                r_row.append(0.0)
        rewards.append(r_row)

    utils = copy.deepcopy(rewards)



    res = copy.deepcopy(problem)
    #Value iteration for eech cell
    for _ in range(ITERATIONS):
        new_utils = copy.deepcopy(utils)
        for i in range(r):
            for j in range(c):
                #Terminal state if G or P
                if (problem.map[i][j]=="G" or problem.map[i][j]=="P"):
                    continue
                mvs = get_pos_moves(problem,i,j)
                #Update util with possible moves, and return best move
                new_utils[i][j],best_move = updateUtil(mvs,problem,rewards,utils,i,j)
                res.map[i][j] = MOVES[best_move]
        utils = new_utils

    # TODO calculate the policy
    policy = Policy(res)
    return policy

def q_solve(problem, iterations):
    """q_solve:  Use Q-learning to find a good policy on an MDP problem.

    Each iteration corresponds to a random drop of the agent onto the map, followed by moving
    the agent until a reward is reached or MAX_MOVES moves have been made.  When an agent
    is sitting on a reward, update the utility of each move from the space to the reward value
    and end the iteration.  (For simplicity, the agent also does this if just dropped there.)
    The agent does not "know" reward locations in its Q-values before encountering the space
    and "discovering" the reward.

    Note that texts differ on when to pay attention to this reward - this code follows the
    convention of scoring rewards of the space you are moving *from*, plus discounted best q-value
    of where you landed.

    Assume epsilon-greedy exploration.  Leave reward letters as-is in the policy,
    to make it more readable.

    Args:
        problem (Problem):  The environment
        iterations (int):  The number of runs from random start to reward encounter
    Returns:
        A Policy for the map
    """
    r = len(problem.map)
    c = len(problem.map[0])
    rewards = []
    qVals = []
    #Initializing Qvals arr and rewards
    for i in range(r):
        q_row = []
        r_row = []
        for j in range(c):
            q_row.append([0.0,0.0,0.0,0.0])
            if problem.map[i][j]=="G":
                r_row.append(GOLD_REWARD)
            elif problem.map[i][j]=="P":
                r_row.append(PIT_REWARD)
            else:
                r_row.append(0.0)
        qVals.append(q_row)
        rewards.append(r_row)

    #Learning
    for _ in range(ITERATIONS):
        #Random drop on the map
        r_drop = random.randint(0,r-1)
        c_drop = random.randint(0,c-1)
        mvs = 0

        while mvs < MAX_MOVES:
            #If we get dropped on p or g, its a terminal state and we end trial.
            if problem.map[r_drop][c_drop] == "P":
                qVals[r_drop][c_drop] = [PIT_REWARD, PIT_REWARD, PIT_REWARD, PIT_REWARD]
                break

            if problem.map[r_drop][c_drop] == "G":
                qVals[r_drop][c_drop] = [GOLD_REWARD, GOLD_REWARD, GOLD_REWARD, GOLD_REWARD]
                break

            pr = random.random()
            possible_moves = get_pos_moves(problem,r_drop,c_drop)
            action = 0
            #WIth prob EXPLORE_PROB we pick a random possible move, otherwise we pick best q
            if pr < EXPLORE_PROB:
                action = random.choice(possible_moves)
            else:
                possibe_qs = [qVals[r_drop][c_drop][x] for x in possible_moves]
                best_action_q = max(possibe_qs)
                action = qVals[r_drop][c_drop].index(best_action_q)

            #Calling role step function using the move we chose to get new coordinates
            new_row,new_col = roll_steps(problem.move_probs,r_drop,c_drop,MOVES[action],r,c)
            #calling new_q to update current q
            q = new_q(rewards,qVals,r_drop,c_drop,new_row,new_col,action)
            #Updating q in qvals arr
            qVals[r_drop][c_drop][action] = q
            #Moving on if not terminal state
            r_drop = new_row
            c_drop = new_col
            mvs+=1


    policy = Policy(problem)

    for i in range (r):
        for j in range(c):
            if problem.map[i][j]=="G" or  problem.map[i][j]=="P":
                continue
            best_action_index = qVals[i][j].index(max(qVals[i][j]))
            best_action = MOVES[best_action_index]
            policy.best_actions[i][j] = best_action


    return policy

#Helper function to return the possible moves from a specific square (to avoid going off map)
def get_pos_moves(problem,r,c):
    moves = []
    if r > 0:
        moves.append(0)
    if r < len(problem.map) - 1:
        moves.append(2)
    if c > 0:
        moves.append(3)
    if c < len(problem.map[0]) - 1:
        moves.append(1)
    return moves

#Helper function to get the reward for a coordinate
def get_reward(problem, r, c):
    if problem.map[r][c] == "G":
        return GOLD_REWARD
    elif problem.map[r][c] == "P":
        return PIT_REWARD
    else:
        return 0.0

#Q learning function uses
def new_q(rewards, utilities, r, c, new_r, new_c, movenum):
    """ Q-learning function.  Returns the new Q-value for space (r,c).
    It's recommended you code and test this before doing the overall Q-learning.

    Should use the LEARNING_RATE and DISCOUNT_FACTOR.

    Args:
        rewards (List[List[float]]):  Reward amounts built into the problem map (indexed [r][c])
        utilities (List[List[List[float]]]):  The Q-values for each action from each space.
                                              (Indexed as [row][col][move])
        r, c (int, int):  Row and column of our location before move
        new_r, new_c (int, int):  Row and column of our location after move
        movenum (int):  Integer index into the Q-values, corresponding to constants UP etc
    Returns:
        float - the new Q-value for the space we moved from
    """
    old_q = utilities[r][c][movenum]
    action_reward = rewards[new_r][new_c]
    best_next_q = max(utilities[new_r][new_c])

    new_q = old_q + LEARNING_RATE * (action_reward + DISCOUNT_FACTOR * best_next_q - old_q)


    return new_q

In [3]:
deterministic_test = """1.0
- - P - -
- - G P -
- - P - -
- - - - -"""

In [4]:
# Notice that we counterintuitively are most likely to go 2 spaces here
very_slippy_test = """0.2 0.7 0.1
- - P - -
- - G P -
- - P - -
- - - - -"""

In [5]:
big_test = """0.6 0.3 0.1
- P - G - P - - G -
P G - P - - - P - -
P P - P P - P - P -
P - - P P - - - - P
- - - - - - - - P G"""

In [6]:
# MDP value iteration tests
print(Problem(deterministic_test).solve(ITERATIONS, False))


D D P R D
R R G P D
U U P D D
U U L L L



In [8]:
print(Problem(sampleMDP).solve(ITERATIONS, False))


D D P R D
R R G P D
U U P D D
U U L L L



In [9]:
print(Problem(very_slippy_test).solve(ITERATIONS, False))


D L P R D
R L G P L
D D P D D
U L U L U



In [10]:
print(Problem(big_test).solve(ITERATIONS, False))


R P R G L P R R G L
P G U P U R U P U U
P P U P P D P D P U
P D U P P D D D L P
R R U L L L L L P G



In [11]:
# Q-learning tests
# Set seed every time for consistent executions;
# comment out to get different random runs
random.seed(340)
print(Problem(deterministic_test).solve(ITERATIONS, True))


R D P R D
R R G P D
U U P D D
U U L L L



In [12]:
random.seed(340)
print(Problem(sampleMDP).solve(ITERATIONS, True))


D D P R D
R R G P D
U U P D D
U U L L L



In [13]:
random.seed(340)
print(Problem(very_slippy_test).solve(ITERATIONS, True))


D L P R D
R L G P L
D L P R D
U L U R U



In [14]:
random.seed(340)
print(Problem(big_test).solve(ITERATIONS, True))


R P R G L P R R G L
P G U P U D U P U U
P P U P P D P D P U
P D U P P R D D L P
R R U L L L L L P G



Once you're done, here are a few thought questions (19 points total):

**3, 5 points) Suppose we are on the deterministic map where there is no sliding on ice, and performing value iteration until it converges.  Supposing 0 < DISCOUNT_FACTOR < 1, how does the policy change if the discount factor changes to another value in that range (or does the policy change at all)?  Why does that happen?  What happens to the policy if DISCOUNT_FACTOR = 1?**

**TODO**


The discount factor affects how much the value of utils/rewards is reduced as time/distance from the reward is increased. A low discount factor will cause the agent to follow paths with more immediate rewards, because paths with several moves until a reward is rached will be exponentially small due to the discount factor being small and being multiplied several times. On the other hand, a high discount factor will favor/allow the agent to explore paths where rewards are far down the line. If discount_facor is 1, the policy  will be wrong (tested in our case above, wrong results given) because upon the first the agent will assume it reached terminal reward states as value iteration updates all adjacent squares next to gold/pits.





**4, 3 points) The value iteration MDP solver updates all squares an equal number of times.  The Q-learner does not.  Which squares might we expect the Q-learner to update the most?**



**TODO**

At the start of the q-learning trial,the agent will explore the grid randomly due to the empty q-table and random drops, however as the learning progresses, the q-learner will tend to move to squares with higher q values. Therfore, we can expect that in the long run the q-learner will update squares in/around the rewards most often.

**5, 11 points) Suppose we change the state information so that, instead of knowing its coordinates on the map, the agent instead knows just the locations of all rewards in a 5x5 square with the agent at the square center.  Thus, at the start of every run, it may not know exactly where it is, but it knows what is in the vicinity.  It also does not know the transition model.**

**a, 2 points) We can't use value iteration here.  Why?**

**b, 4 points) How many state-action combinations are possible, assuming the contents of the agent's own square don't matter, and every other square could have a pit, gold, or an empty square as in the example maps?  Is a lookup table of Q-values feasible if we allocate memory for each possible state-action combination?  (Let's define "feasible" as "able to be stored in a gig or less of memory," assuming 64-bit values.)**

**c, 5 points) Let's suppose we want to instead generate Q-values with a classic neural network with a single hidden layer.  The inputs are the contents of the 24 squares in the 5x5 square that the player is not  in (we can encode gold = 1, nothing = 0, pit = -1).  There are 10 hidden units.  There are 4 output units corresponding to the 4 possible actions' Q-values.  How much memory is required for the weights of this network, assuming each is a 32-bit float (don't forget bias weights for each unit)?  Comparing to part (b), is it more efficient in memory to use a lookup table for Q(s,a), or this neural network?**


**a) TODO**

value iteration requires us to know the rewards/state of the entire grid, not jus the surrounding squares. Value iteration is not possible in this scenario because once we try to update the squares at the endge of our 5x5, we don't have information about the squares outside of it so we cannot apply bellmans equation.

**b) TODO**

Since there are 24 squares in the 5x5 excluding the square we are in and 3 possible rewards in each square, this gives us

3*3*3*... = 3^24 reward layouts/state.

Since there are 4 actions, this gives us 3^24*4 state-action combinations. if each combination is 64-bits, this gives us

 3^24*4*64 = 7.2301961e+13 bits = 9037.745 gigabytes. therefore it is unfeasible.



**c) TODO**

in the neural network case we have (25-1 = 24) input nodes, 10 hidden nodes, and 4 output nodes (each action).
The total number of weights is:
24*10(input-hidden)+10(bias) = 250
10*4(hidden-output)+4(bias) = 44
total = 294

Since each weight is 32-bits, the total number of bits is:

294*32 = 9408bits, which is singificantly less than a gig. Comparing with part-b, it seems much more efficient to use a nearual network instead of a full q-lookup.


**Remember to submit your code on Blackboard as both an .ipynb (File->Download->.ipynb) and a PDF (Print->Save as PDF).**